In [ ]:
%pip install playwright databricks-sdk
%restart_python

In [ ]:
import re

from databricks.sdk import WorkspaceClient


def strip_scheme(host_or_url: str) -> str:
    return host_or_url.replace("https://", "").replace("http://", "").strip("/")


dbutils.widgets.text("app_url", "<app-name> OR <full-app-host-or-url>")
dbutils.widgets.text("scope", "")
dbutils.widgets.text("api_path", "/api/sample")
dbutils.widgets.text("expected_api_message", "Hello from API sample")

raw_app_url = dbutils.widgets.get("app_url").strip()
scope = dbutils.widgets.get("scope").strip()
expected_api_message = dbutils.widgets.get("expected_api_message").strip()

client_id = dbutils.secrets.get(scope=scope, key="client_id")
client_secret = dbutils.secrets.get(scope=scope, key="client_secret")

workspace = WorkspaceClient(client_id=client_id, client_secret=client_secret)

auth_headers = workspace.config.authenticate()
def resolve_app_host(app_url: str, workspace_host: str) -> str:
    app_host = strip_scheme(app_url)
    if not app_host:
        raise ValueError("app_url is required")
    if "." in app_host:
        return app_host

    workspace_host = strip_scheme(workspace_host)
    m = re.fullmatch(r"adb-(\d+)\.(\d+)\.azuredatabricks\.net", workspace_host)
    if not m:
        raise ValueError(f"Unsupported workspace host format: {workspace_host}")

    workspace_id, shard = m.groups()
    return f"{app_host}-{workspace_id}.{shard}.azure.databricksapps.com"

raw_api_path = dbutils.widgets.get("api_path").strip()
api_path = "/" + raw_api_path.lstrip("/")
if not api_path.startswith("/api/"):
    raise ValueError(f"api_path must start with '/api/': {api_path}")

workspace_host = spark.conf.get("spark.databricks.workspaceUrl")
app_host = resolve_app_host(raw_app_url, workspace_host)

base_url = f"https://{app_host}".rstrip("/")
url = f"{base_url}{api_path}"

items = workspace.workspace.list(path="/")
print("Authentication successful. Workspace root contents:")
for item in items:
    print(f"- {item.path} ({item.object_type})")

browser_headers = {}
if "Authorization" in auth_headers:
    browser_headers["Authorization"] = auth_headers["Authorization"]



In [ ]:
from playwright.async_api import Error as PlaywrightError
from playwright.async_api import async_playwright

async def run_playwright_test():
    async with async_playwright() as p:
        try:
            browser = await p.chromium.launch()
        except PlaywrightError as exc:
            raise RuntimeError(f"Chromium launch failed: {exc}") from exc
        try:
            context = await browser.new_context(extra_http_headers=browser_headers)
            page = await context.new_page()
            response = await page.goto(url, wait_until="domcontentloaded")

            if response is None:
                raise RuntimeError("Navigation completed without an HTTP response.")

            if response.status in {401, 403}:
                raise AssertionError(
                    f"Authentication failed with HTTP {response.status}. "
                    "Check scope and client secrets."
                )
            if response.status != 200:
                raise AssertionError(
                    f"Expected HTTP 200 from {url}, got {response.status} {response.status_text}"
                )

            content_type = response.headers.get("content-type", "")
            if "application/json" not in content_type.lower():
                raise AssertionError(
                    f"Expected JSON response content-type, got {content_type!r}"
                )

            payload = await response.json()
            assert payload.get("status") == "ok", (
                f"Expected payload status='ok', got {payload.get('status')!r}"
            )
            assert payload.get("message") == expected_api_message, (
                f"Expected message={expected_api_message!r}, got {payload.get('message')!r}"
            )
            assert payload.get("path") == api_path, (
                f"Expected path={api_path!r}, got {payload.get('path')!r}"
            )
        finally:
            await browser.close()

await run_playwright_test()
print("Test passed!")
